In [1]:
import os
from moviepy import editor
from tqdm.notebook import tqdm_notebook

In [6]:
def get_time(timestamp, e=0, max_t=None):
    if e>0 and max_t is None:
        print("Please provide max_t")
        return
    if ":" in timestamp:
        minute = int(timestamp.split(":")[0])*60
        second = float(timestamp.split(":")[1])
        t = minute + second
        return max(t+e, 0) if e<0 else min(t+e, max_t)
    else:
        try:
            return max(float(timestamp)+e, 0) if e<0 else min(float(timestamp)+e, max_t)
        except Exception as e:
            print(e)

In [7]:
def get_filename(file, n):
    ext = "."+file.split('.')[-1]
    new_name = file[:-len(ext)] + "_" + str(n) + ext
    return new_name

In [11]:
def extract_clips(data_dir, root="Data", output_root="Processed-Videos", gap=2, skip_classes=[]):
    base_path = os.path.join(root, data_dir)
    if not os.path.exists(base_path):
        print(f"{base_path} does not exists.")
        return
    subdirs = []
    for d in os.listdir(base_path):
        if os.path.isdir(os.path.join(base_path, d)):
            subdirs.append(d)
    for d in subdirs:
        if d in skip_classes:
            continue
        os.mkdir(os.path.join(root, output_root, d))
        file = open(os.path.join(base_path, f"{d}_Labels.txt"), mode='r', encoding='utf-8')
        lines = file.readlines()
        print(f"Processing {d} category videos:")
        for line in tqdm_notebook(lines):
            l = line.strip().split()
            vid_name = l[0]
            vid_path = os.path.join(base_path, d, vid_name)
            if os.path.exists(vid_path):
                video = editor.VideoFileClip(vid_path)
                if len(l[1:])%2 != 0:
                    print(f"Timestamps are not in complete pairs. Ignoring last timestamp for video {os.path.join(base_path, d, vid_name)}")
                anomaly_timestamps = []
                # Anamoly part extraction
                n_pairs = len(l[1:])//2
                for n in range(n_pairs):
                    new_vid = f"{os.path.join(root, output_root, d, get_filename(vid_name, n))}"
                    t1, t2 = get_time(l[n*2+1], e=-0.7), get_time(l[n*2+2], e=0.7, max_t=video.duration)
                    if t1 is not None and t2 is not None:
                        clip = video.subclip(t1, t2)
                        clip.write_videofile(new_vid, logger=None)
                        anomaly_timestamps.append((t1, t2))
                    else:
                        print(f"Skipping timestamps pairs {l[n*2+1]} and {l[n*2+2]} for {vid_path}")
                # Normal part extraction
                normal_output_dir = os.path.join(root, output_root, "Normal")
                if not os.path.exists(normal_output_dir):
                    os.mkdir(normal_output_dir)
                normal_timestamps = []
                if len(anomaly_timestamps) > 0:
                    if anomaly_timestamps[0][0]-gap > 0:   # gap b/w normal & anomaly
                        normal_timestamps.append((0, anomaly_timestamps[0][0]-gap))
                    for i in range(len(anomaly_timestamps)-1):
                        start = anomaly_timestamps[i][1]+gap
                        end = anomaly_timestamps[i+1][0]-gap
                        if end-start > 0:
                            normal_timestamps.append((start, end))
                    if anomaly_timestamps[-1][1]+gap < video.duration:
                        normal_timestamps.append((anomaly_timestamps[-1][1]+gap, video.duration))
                for i, t in enumerate(normal_timestamps):
                    if t[1]-t[0]>=1.3:
                        clip = video.subclip(t[0], t[1])
                        new_vid = f"{os.path.join(normal_output_dir, get_filename('Normal_'+vid_name, i))}"
                        clip.write_videofile(new_vid, logger=None)
            else:
                print(f"{vid_path} doesnot exists. Skipping...")

In [9]:
extract_clips("Anomaly-Videos-Part-3")

Processing RoadAccidents category videos:


  0%|          | 0/149 [00:00<?, ?it/s]

Processing Robbery category videos:


  0%|          | 0/150 [00:00<?, ?it/s]

Processing Shooting category videos:


  0%|          | 0/50 [00:00<?, ?it/s]

In [10]:
extract_clips("Anomaly-Videos-Part-2")

Processing Burglary category videos:


  0%|          | 0/106 [00:00<?, ?it/s]

Processing Explosion category videos:


  0%|          | 0/51 [00:00<?, ?it/s]

IndexError: list index out of range

In [12]:
extract_clips("Anomaly-Videos-Part-2", skip_classes=["Burglary", "Explosion"])

Processing Fighting category videos:


  0%|          | 0/58 [00:00<?, ?it/s]

In [13]:
extract_clips("Anomaly-Videos-Part-1")

Processing Abuse category videos:


  0%|          | 0/50 [00:00<?, ?it/s]

Processing Arson category videos:


  0%|          | 0/48 [00:00<?, ?it/s]

Processing Assault category videos:


  0%|          | 0/50 [00:00<?, ?it/s]